# 1. Introducción
Tras haber obtenido los datos y procesarlos, pasamos a visualizarlos y hacer algunos análisis a partir únicamente de ellos.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from pathlib import Path
from scipy.stats import norm

In [2]:
import sys, os
print("PY:", sys.executable)

PY: /Users/jenriquezafra/Proyectos/Dev/python/Chaos_Signals/.venv/bin/python


---
# 2. Carga de datos
Obtenemos los datos procesados.

In [3]:
# obtenemos la ruta
dir_path = Path('../data').resolve()
path = dir_path / 'processed'
dfs = {}

# obtenemos todos los files y los metemos en un diccionario
for file in path.iterdir():
    if file.suffix == ".parquet":
        name = file.stem.split("_")[0]
        dfs[name] = pd.read_parquet(file, engine='pyarrow')


In [4]:
# accedemos a uno para comprobar
print(dfs['AMD'].head())

                               close       high        low       open  \
date                                                                    
2022-08-23 00:00:00+02:00  92.489998  94.440002  92.110001  92.389999   
2022-08-24 00:00:00+02:00  92.730003  93.379997  90.900002  92.209999   
2022-08-25 00:00:00+02:00  97.180000  97.570000  93.139999  93.139999   
2022-08-26 00:00:00+02:00  91.180000  97.599998  91.120003  96.290001   
2022-08-29 00:00:00+02:00  88.489998  91.190002  88.260002  90.050003   

                             volume  daily_return  log_return     range  \
date                                                                      
2022-08-23 00:00:00+02:00  52927000     -0.003770   -0.003777  2.330002   
2022-08-24 00:00:00+02:00  56520400      0.002595    0.002592  2.479996   
2022-08-25 00:00:00+02:00  61016200      0.047989    0.046873  4.430000   
2022-08-26 00:00:00+02:00  65552500     -0.061741   -0.063729  6.479996   
2022-08-29 00:00:00+02:00  61142500   

---
# 3. Análisis univariado
Vamos a estudiar las series temporales de precios y volumen, histogramas de returns, y boxplots para ver outliers.

## 3.1. Series temporales

In [5]:
fig_close = go.Figure()

for name, df in dfs.items():
    fig_close.add_trace(
        go.Scatter(
            x=df.index,
            y=df['close'],
            mode='lines',
            name=name,
        )
    )

fig_close.update_layout(title='Precio al cierre', xaxis_title='Date', yaxis_title='Open')
fig_close.show()

In [6]:
fig_ret = go.Figure()

for name, df in dfs.items():
    fig_ret.add_trace(
        go.Scatter(
            x=df.index,
            y=df['daily_return'],
            mode='lines',
            name=name,
        )
    )

fig_ret.update_layout(title='Daily Return ', xaxis_title='Date', yaxis_title='Return')
fig_ret.show()

Esta visualización tiene como ventaja que todo está normalizado. En precios algunas son mucho más grandes que otras y no se ven bien. 

El daily_return nos permite ver épocas de volatilidad.

In [7]:
fig_vol = go.Figure()

for name, df in dfs.items():
    fig_vol.add_trace(
        go.Scatter(
            x=df.index,
            y=df['volume'],
            mode='lines',
            name=name,
        )
    )

fig_vol.update_layout(title='Volúmenes ', xaxis_title='Date', yaxis_title='Volume')
fig_vol.show()

## 3.2. Histogramas
Haremos de daily_return, de rango, volúmenes y un log-return vs. normal gaussiana (opcional)

In [8]:
fig_hist_ret = go.Figure()

for name, df in dfs.items():
    fig_hist_ret.add_trace(
        go.Histogram(
            x=df['daily_return']*100, # convertimos a porcentaje
            name=name,
            opacity=0.6,
        )
    )

fig_hist_ret.update_layout(
    title='Daily Return histogram',
    xaxis_title='Daily Return [%]', 
    yaxis_title='Count')

fig_hist_ret.show()

In [37]:
fig_hist_vol = go.Figure()

for name, df in dfs.items():
    fig_hist_vol.add_trace(
        go.Histogram(
            x=df['volume'],
            name=name,
            opacity=0.6,
        )
    )

fig_hist_vol.update_layout(
    title='Volume histogram',
    xaxis_title='Volume', 
    yaxis_title='Count',
    )

fig_hist_vol.show()

Para los log_returns, por ahora lo haré para un solo stock,

In [9]:
df_amd = dfs['AMD']
log_returns = df_amd['log_return']

# creamos la normal de referencia ajustada a nuestros datos
mu = log_returns.mean()
sigma = log_returns.std()
x = np.linspace(log_returns.min(), log_returns.max(), 200)
pdf = norm.pdf(x, mu, sigma)

# hacmeos el gráfico
fig_log_ret = go.Figure()

fig_log_ret.add_trace(
    go.Histogram(
        x=log_returns,
        histnorm='probability density',
        name='Log Returns',
        opacity=0.6,
    )
)

# añadimos la normal de referencia
fig_log_ret.add_trace(
    go.Scatter(
        x=x,
        y=pdf,
        mode='lines',
        name='Normal Distribution',
        line=dict(color='red', width=2)
    )
)

fig_log_ret.update_layout(
    title='Log Returns Distribution with Normal Fit [AMD]',
    xaxis_title='Log Returns',
    yaxis_title='Probability Density',
    legend=dict(x=0.01, y=0.99, traceorder='normal', orientation='h')
)

fig_log_ret.show()


- En este caso vemos que están muy concentrados alrededor de 0 (lo esperado).

- Pico más alto en el 0 (en general retornos bajos)

- Colas más grandes que la gaussiana (mayores eventos extremos)

- Asimetría: parece cola hacia la izquierda (negativa)

## 3.3. Boxplots
Se deja para más adelante. Ahora no es relevante.

---
# 4. Análisis multivariado

## 4.1. Correlaciones
Vemos matriz de correlación de retornos, heatmap y correlación rolling (ventanas de 60-90 días) entre pares relevantes.
Aquí voy a empezar a tomar como target a AMD y comparar con peers. Concretamente contra NVDA (lead), MSFT (tech), y QQQ (benchmark).

In [12]:
# hacemos un df con los retornos logarítmicos de todos los activos
log_returns = pd.concat(
    {ticker: dfs[ticker]['log_return'] for ticker in dfs.keys()},
    axis=1
)

# matriz de correlación
corr = log_returns.corr()

fig = go.Figure(
    data=go.Heatmap(
        z=corr.values,
        x=corr.columns,
        y=corr.index,
        colorscale='RdBu',
        zmin=-1, zmax=1,
#        text=np.round(corr.values, 2),
#        texttemplate="%{text}",
#        textfont={"size":14},
        colorbar=dict(title='Correlation')
    )
)

# layout
fig.update_layout(
    title='Matriz de correlación de log-returns',
    width=600, height=600
)
fig.show()

Esto permite ver rápidamente los tickers más importantes. Los FX son insignificantes; de las commodities solo afecta un poco el cobre (HG=F); las tech e índices son todos son importantes. El que más correlación tiene es el índice SOXX (semiconductores), y el que tiene más correlación inversa es VIX (volatilidad).

In [13]:
# --- Config ---
target = "AMD"
peers  = ["NVDA", "MSFT", "QQQ"]
window = 20  # usa 5 para 1 semana, 20 para ~1 mes

# --- Unificar log-returns desde tu dict de dataframes ---
# Cada df de dfs[ticker] debe tener la columna 'log_return' y un índice datetime
logrets = pd.concat(
    {t: dfs[t]["log_return"].rename(t) for t in [target] + peers},
    axis=1
).dropna(how="all")

# --- Calcular rolling correlations AMD vs cada peer ---
rollcorr = pd.DataFrame(index=logrets.index)
for p in peers:
    # alinear y calcular correlación móvil
    rollcorr[f"{target} vs {p}"] = (
        logrets[target]
        .rolling(window)
        .corr(logrets[p])
    )

rollcorr = rollcorr.dropna(how="all")

# --- Plotly (go) ---
fig = go.Figure()

for col in rollcorr.columns:
    fig.add_trace(go.Scatter(
        x=rollcorr.index,
        y=rollcorr[col],
        mode="lines",
        name=col
    ))

# línea horizontal en 0 para referencia
fig.add_hline(y=0, line_width=1, line_dash="dash", opacity=0.6)

fig.update_layout(
    title=f"Rolling correlation de log-returns — {target} vs peers (ventana={window})",
    xaxis_title="Fecha",
    yaxis_title="Correlación",
    yaxis=dict(range=[-1, 1]),
    legend_title="Pares",
    width=900, height=450
)

fig.show()

## 4.2. Análisis de dimensionalidad
PCA sobre retornos. Scatter de PC1 vs. PC2. Loadings de cada activo en cada componente.

## 4.3. Relaciones no lineales